In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!pip install fasttext
!pip install q keras==2.2.5

In [0]:
%tensorflow_version 1.x
# %tensorflow_version 2.x

import sys
import os
import time
import multiprocessing
import numpy as np
from collections import namedtuple
import _pickle as pickle # object serialization
import csv
try:
    csv.field_size_limit(sys.maxsize)
except:
    import unicodecsv
    import ctypes
    unicodecsv.field_size_limit(int(ctypes.c_ulong(-1).value // 2))
import re
import time

import tensorflow as tf
print(tf.version.VERSION)
from tensorflow import keras

import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.models import Model, Sequential
from keras.layers.convolutional import MaxPooling1D, Convolution1D
from keras.layers.recurrent import LSTM
from keras.optimizers import Adam

import glob
import stat

from scipy.sparse import lil_matrix
import pickle as pkl
import pandas as pd

from datetime import datetime
from multiprocessing import Queue, Process

import gensim

from sklearn import utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# from tensorflow.contrib import learn

import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import coverage_error
from sklearn.metrics import classification_report

from keras.callbacks import Callback

import collections
from copy import deepcopy
from tqdm import tqdm

from gensim.models import Word2Vec
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from gensim.corpora.dictionary import Dictionary

import fasttext

from collections import namedtuple

from gensim.models.doc2vec import LabeledSentence

from keras.models import load_model
from keras.callbacks import Callback
from keras.callbacks import EarlyStopping
from keras.callbacks import LearningRateScheduler

1.15.0


Using TensorFlow backend.


In [0]:
# folder helper
def get_list_files(path, extension):
    if extension:
        return glob.glob(path + '*.' + extension)
    return glob.glob(path)

def create_folder(destination_path):
    if not os.path.exists(destination_path):
        os.makedirs(destination_path)

def link_paths(root_path, ending_path):
    return os.path.join(root_path, ending_path)

def join_paths(root_path, ending_path):
    path = link_paths(root_path, ending_path)
    create_folder(path)
    return path

def get_root_location(ending_path):
    return join_paths('/content/drive/My Drive/Colab Notebooks/', ending_path)

def ensure_exists_path_location(path):
    if os.path.exists(path):
        return True
    return False

In [0]:
# txt helper
def handle_partial_args(source_path):
    source_path = link_paths(source_path, '*')
    source_path = get_list_files(source_path, None)
    source_path = list(map(lambda path : path + '/', source_path))
    print("source path: %s" % source_path)

    if len(source_path) == 0 or source_path[len(source_path)-1][-5:-1] == '.xml':
        return th.source_path_warnings()
    else:
        folder_level = source_path[0].count('/')-1
        print("folder destination level: %s" % folder_level)
        return source_path, folder_level

def get_txt_text(file, length):
    for index in range(length):
        text = file.readline()
    return text

def fill_dataframe(data_frame, classifications_df, id_, classcode, abstract, claim, description):
    if classcode != "":
        # shrink the set to only_top_classes? TRUE/FALSE
        classcode = cut_down(classcode, 4, ['H', 'B', 'C'], False) # H, B, C
        data_frame.loc[data_frame.shape[0] + 1] = [id_, abstract, claim, description, classcode]
        classifications_df = calculate_class_distribution(classcode, classifications_df)

def handle_patent_file(data_frame, classifications_df, path_filename):
    file = open(path_filename, "r")
    id_ = get_patent_id(path_filename)
    kind = get_txt_text(file, 1).strip()
    if kind == 'A1':
        classcode = get_txt_text(file, 1).strip()
        applicant = get_txt_text(file, 1).strip()

        abstract = get_txt_text(file, 1).strip()
        citations = get_txt_text(file, 1).strip()
        file.close()

        fill_dataframe(data_frame, classifications_df, id_, classcode, abstract, None, None)
    elif kind == 'B1':
        classcode = get_txt_text(file, 1).strip()
        id_respective_document = get_txt_text(file, 1).strip()

        # abstract = get_txt_text(file, 1).strip()
        claim = get_txt_text(file, 1).strip()
        description = get_txt_text(file, 1).strip()
        # citations = get_txt_text(file, 1).strip()
        file.close()

        fill_dataframe(data_frame, classifications_df, id_, classcode, None, claim, description)
    else:
        if kind == 'A1B1':
            print("eu_mix_patent !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        else:
            print("us_patent     !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        classcode = get_txt_text(file, 1).strip()
        applicant = get_txt_text(file, 1).strip()

        abstract = get_txt_text(file, 1).strip()
        claim = get_txt_text(file, 1).strip()
        description = get_txt_text(file, 1).strip()
        citations = get_txt_text(file, 1).strip()
        file.close()

        fill_dataframe(data_frame, classifications_df, id_, classcode, abstract, claim, description)
    return id_

def handle_path_patent(data_frame, classifications_df, path):
    return list(map(lambda path_filename : handle_patent_file(data_frame, classifications_df, path_filename), get_list_files(path, 'txt')))

def load_data(source_path):
    print('###  reading patents  ###')
    """ load_data """
    data_frame = pd.DataFrame(columns=['abstract', 'claim', 'description', 'classification'])
    classifications_df = pd.DataFrame(columns=['class', 'count'])
    patent_ids = []

    patent_ids = list(map(lambda path : handle_path_patent(data_frame, classifications_df, path), tqdm(source_path)))
    patent_ids = get_flat_list(patent_ids)

    data_frame['id'] = data_frame.index
    classifications_df.sort_values(by=['count'], ascending=False, inplace=True, kind='quicksort')
    return patent_ids, data_frame, classifications_df

def handle_row(row, ids_list):
    if isinstance(row, pd.Series):
        try:
            id_, patent_id, text, class_ = row.tolist()
        except:
            patent_id, text, class_ = row.tolist()
        tokens = tokenize_text(text)
        if len(tokens) < 2:
            ids_list.append(patent_id) 

def check_out_for_whitespaces(text):
    if isinstance(text, str):
        return ' '.join([element for element in tokenize_text(text) if len(element) > 2 and len(element) < 31])

def check_out_empty_texts_and_wrong_classcodes(data_frame):
    print('dataframe shape: ', data_frame.shape)
    ids_list = []
    data_frame.apply(lambda row : handle_row(row, ids_list), axis=1)  
    data_frame.set_index(data_frame['patent_id'], inplace=True)
    
    data_frame.drop(ids_list, axis=0, inplace=True)

    data_frame['text'] = data_frame['text'].apply(lambda text : check_out_for_whitespaces(text))
    print('ids_list: ', ids_list)
    print('dataframe shape: ', data_frame.shape)
    return data_frame

def clean_text(text):
    text = re.sub(r'\b\w{1,2}\b', '', text)
    text = " ".join(text.split())
    return text

def get_final_df(patent_ids, temp_df, classif_type):
    classification_target = 'description_claim_abstract_title'

    data_frame = pd.DataFrame(columns=['id_', 'patent_id', 'text', 'classification'])

    if temp_df.shape[1] == 5:
        data_frame['patent_id'] = temp_df['patent_id']
    else:
        data_frame["patent_id"] = patent_ids

    # textual_information = temp_df["abstract"]
    # textual_information = temp_df["claim"]
    # textual_information = temp_df["description"]
    textual_information = temp_df["abstract"].str.cat(temp_df["claim"], sep =" ", na_rep=" ").str.cat(temp_df["description"], sep =" ", na_rep=" ")
    data_frame['text'] = textual_information

    # data_frame['text'] = data_frame['text'].apply(lambda text : clean_text(text))
    data_frame = check_out_empty_texts_and_wrong_classcodes(data_frame)

    classification_types = get_classifications(temp_df)
    data_frame["classification"] = classification_types[classif_type]
    return data_frame, classification_target, classif_type

In [0]:
# tool helper
def get_flat_list(list_of_lists):
    return [element for elements in list_of_lists for element in elements]

def tokenize_text(text):
    return text.split()

def get_string_from_list(list_, linking_string):
    return linking_string.join(element for element in list_ if isinstance(element, str))

def cut_down(classification, index, top_classes, only_top_classes):
    temp_list = tokenize_text(classification)
    if only_top_classes:
        new_classification_list = list(set(filter(lambda element : element[:index] in top_classes, temp_list)))
        new_classification = get_string_from_list(new_classification_list, ' ')
    else:
        new_classification_list = list(set(map(lambda element : element[:index], temp_list)))
        new_classification = get_string_from_list(new_classification_list, ' ')
    return new_classification

def calculate_class_distribution(classification, classifications_df):
    if isinstance(classification, str):
        for _class in tokenize_text(classification):
            if classifications_df['class'].str.contains(_class).any():
                index = classifications_df.index[classifications_df['class'] == _class]
                classifications_df.loc[index[0], ['count']] += 1
            else:
                classifications_df.loc[classifications_df.shape[0] + 1] = [_class, 1]
        return classifications_df

def tokenize_complex_text_in_set(text):
    return set(map(lambda word : word, tokenize_text(text)))

def get_set_from_index(index_set, data_set):
    set_ = np.ndarray(shape=(index_set.shape[0], data_set.shape[1]))
    for index, data_index in enumerate(index_set['text'].values):
        set_[index-1] = data_set[data_index-1]
    return set_, index_set['patent_id']

In [0]:
# classification helper
def get_train_test_from_data(X, Y):
    return train_test_split(X, Y, random_state=0, test_size=.2, shuffle=True)

def get_train_test_from_dataframe(data_frame):
    return train_test_split(data_frame, random_state=0, test_size=.2, shuffle=True)

def get_distinct_class_substrings(classification_list, first_int, second_int):
    list_ = []
    for element in classification_list:
        for class_ in tokenize_text(element):
            if class_[first_int:second_int] not in list_:
                list_.append(class_[first_int:second_int])
    list_.sort()
    return list_

def get_class_substrings(classification_list, first_int, second_int):
    list_ = []
    for element in classification_list:
        string = ""
        for class_ in tokenize_text(element):
            if class_[first_int:second_int] not in string:
                string += class_[first_int:second_int] + ' '
        list_.append(string)
    return list_

def get_classifications(temp_df):
    ## Load utility data
    classifications = temp_df['classification'].tolist()

    valid_sections = get_distinct_class_substrings(classifications, 0, 1)
    valid_classes = get_distinct_class_substrings(classifications, 1, 3)
    valid_subclasses = get_distinct_class_substrings(classifications, 3, 4)

    sections = get_class_substrings(classifications, 0, 1)
    classes = get_class_substrings(classifications, 0, 3)
    subclasses = get_class_substrings(classifications, 0, 4)

    classification_types = {
        "sections": sections,
        "classes": classes,
        "subclasses": subclasses
    }
    return classification_types

def save_training_set(training_set, model_name):
    print('###  saving_training_set  ###')
    root_location = get_root_location('data/saved_training_set/')
    model_name = model_name.replace("/", "-")
    path = link_paths(root_location, 'training '+model_name+' '+str(datetime.now())[:-10]+'.npy')
    np.save(path, training_set)

def save_sets(sets_location, train_data, test_data, val_data, train_labels, test_labels, val_labels, settings):
    try:
        date = '01-01-2020'
        actual_sets_location = join_paths(sets_location, date)
        with open(link_paths(actual_sets_location, 'training_set '+date+'.pkl'), "wb") as f:
            pkl.dump([train_data, train_labels], f)
        with open(link_paths(actual_sets_location, 'testing_set '+date+'.pkl'), "wb") as f:
            pkl.dump([test_data, test_labels], f)
        with open(link_paths(actual_sets_location, 'validation_set '+date+'.pkl'), "wb") as f:
            pkl.dump([val_data, val_labels], f)
        with open(link_paths(actual_sets_location, 'settings '+date+'.pkl'), "wb") as f:
            pkl.dump(settings, f)
    except:
        print('A problem occurred while saving the sets!')

def load_sets(sets_location, date='01-01-2020'):
    try:
        actual_sets_location = join_paths(sets_location, date)
        with open(link_paths(actual_sets_location, 'training_set '+date+'.pkl'), "rb") as f:
            train_data, train_labels = pkl.load(f)
        with open(link_paths(actual_sets_location, 'testing_set '+date+'.pkl'), "rb") as f:
            test_data, test_labels = pkl.load(f)
        with open(link_paths(actual_sets_location, 'validation_set '+date+'.pkl'), "rb") as f:
            val_data, val_labels = pkl.load(f)
        with open(link_paths(actual_sets_location, 'settings '+date+'.pkl'), "rb") as f:
            settings = pkl.load(f)
            # classes, n_classes, vocab_processor, len_vocabulary = pkl.load(f)
        return train_data, test_data, val_data, train_labels, test_labels, val_labels, settings
    except:
        print('A problem occurred while loading the sets!')
        return None, None, None, None, None, None, None

def save_data_frame(script_key, data_frame, csvfile):
    path_to_csv = get_root_location('model_dataframe/')
    date = datetime.now().isoformat()
    if csvfile:
        output_path = fh.link_paths(path_to_csv, csvfile)
    else:
        output_path = fh.link_paths(path_to_csv, 'dataframe '+date+'.csv')

    data_frame.to_csv(output_path, index=False, sep=',', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

def load_data_frame(script_key, csvfile, date='01-01-2020'):
    path_to_csv = get_root_location('model_dataframe/')
    if csvfile:
        input_path = link_paths(path_to_csv, csvfile)
    else:
        input_path = link_paths(path_to_csv, 'dataframe '+date+'.csv')

    data_frame = pd.read_csv(input_path, sep=',', quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ", header=None, engine='python')
    data_frame.columns = ['patent_id', 'text', 'classification']
    
    # data_frame['text'] = data_frame['text'].apply(lambda text : clean_text(text))
    data_frame = check_out_empty_texts_and_wrong_classcodes(data_frame)
    
    classification_df = pd.DataFrame(columns=['class', 'count'])
    # data_frame['classification'].apply(lambda class_ : calculate_class_distribution(class_, classification_df))
    return data_frame, classification_df

def apply_simple_binarizer(classification, classes):
    print('###  label_binarizer  ###')
    binarized_classification = label_binarize(classification, classes=classes)
    return binarized_classification, classes, binarized_classification.shape[1]

# TODO: may be the same as the simple
def apply_label_binarizer(classification):
    lb = LabelBinarizer()
    return lb.fit_transform(classification)

def apply_multilabel_binarizer(data_frame):
    print('###  multi_label_binarizer  ###')
    ################################################ classification: from text to sparse binary matrix [[0, 1, 0],[1, 0, 1]]
    temp_classification = data_frame.apply(lambda row : tokenize_complex_text_in_set(row['classification']), axis=1)
    df_to_list = temp_classification.tolist()
    mlb = MultiLabelBinarizer()
    mlb.fit(df_to_list)
    classes = list(mlb.classes_)
    return mlb.transform(df_to_list), classes, len(classes)

def apply_tfidf_vectorizer_fit_transform(data_frame):
    # list of strings, which are consequently made of words (no stopwords, stemming already applied ...) - preprocessed text
    print('###  tfidf_vectorizer_with_lambda  ###')
    vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2')
    return vectorizer.fit_transform(data_frame['text'].apply(lambda x : np.str_(x))), data_frame['patent_id'], vectorizer

def apply_tfidf_vectorizer_fit(data_frame):
    print('###  tfidf_vectorizer_text  ###')
    vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2')
    vectorizer.fit(data_frame['text'].apply(lambda x : np.str_(x)))
    return vectorizer

def apply_tfidf_vectorizer_transform(text, vectorizer):
    return vectorizer.transform(text.apply(lambda x : np.str_(x)))

def apply_word2vec_word_averaging(data_frame):
    print('###  word2vec_word_averaging_vectorizer  ###')
    path_ = current_path[:current_path.rfind('/', 0, -8)] + "/data/GoogleNews-vectors-negative300.bin"
    print("google vectors : ", path_)
    try:
        wv = gensim.models.KeyedVectors.load_word2vec_format(path_, binary=True)
    except:
        print("unable to find word2vec model from google, downloading...")
        wv = gensim.models.KeyedVectors.load_word2vec_format("https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz", binary=True)
        print("done...")
    wv.init_sims(replace=True)
    helper_word2vec = Word2VecHelper()

    text_tokenized = data_frame.apply(lambda row : helper_word2vec.w2v_tokenize_text(row['text']), axis=1).values
    return helper_word2vec.word_averaging_list(wv, text_tokenized)

def apply_doc2vec(data_frame, type_):
    Y, classes, n_classes = apply_classification_vectorizer(type_, data_frame)
    print('finisched class vect')
    X_train, X_test, y_train, y_test = get_train_test_from_data(data_frame, Y)
    print('finisched get train test')
    patent_ids = X_train['patent_id']
    print('finisched ids')
    helper_doc2vec = Dov2VecHelper()
    print('finisched doc2vec helpers')
    X_train = helper_doc2vec.label_sentences(X_train['text'], 'Train')
    print('finisched label sent')
    X_test = helper_doc2vec.label_sentences(X_test['text'], 'Test')
    print('finisched label sent')
    all_data = X_train + X_test
    print('finisched all data')
    model_dbow = train_doc2vec(all_data)
    print('finisched train doc2vec')
    train_vectors_dbow = helper_doc2vec.get_vectors(model_dbow, len(X_train), 300, 'Train')
    print('finisched get vect')
    test_vectors_dbow = helper_doc2vec.get_vectors(model_dbow, len(X_test), 300, 'Test')
    print('finisched get vect')
    return train_vectors_dbow, test_vectors_dbow,  y_train, y_test, classes, n_classes, patent_ids

def apply_standard_vectorizer(data_frame, type_):
    Y, classes, n_classes = apply_classification_vectorizer(type_, data_frame)
    X, vocab_processor, len_vocabulary = apply_vocabulary_processor(data_frame['text'])
    # a list of the words used in the text, identified by a unique number for each different word

    X_train, X_test, y_train, y_test = get_train_test_from_data(X, Y)

    print("Vocabulary Size: {:d}".format(len_vocabulary))
    print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_test)))
    return X_train, X_test, y_train, y_test, classes, n_classes, vocab_processor, len_vocabulary

def set_string_for_fasttext(item):
    return str(item).replace(' or ', ' ').replace(', or ', ' ').replace(',', ' ').replace(',', '__label__').replace('$$', ' ').replace('$', ' ').replace(' ', ' __label__').replace('___', '__')

def apply_classification_vectorizer(type_, data_frame):
    classification = data_frame['classification']
    if type_ == 'simple':
        classes = ['H', 'B', 'C'] # useful if i need to shrink the set of classes
        return apply_simple_binarizer(classification, classes)
    elif type_ == 'label_binarizer':
        return apply_label_binarizer(classification), None, 0
    elif type_ == 'multi_label':
        return apply_multilabel_binarizer(data_frame)
    elif type_ == 'fasttext':
        return ['__label__'+set_string_for_fasttext(item) for item in data_frame['classification']], None, 0
    return classification, None, 0

def apply_df_vectorizer(data_frame, type_, classification_type, model_name):
    # type_ = "doc2vec"
    # classification_type = "multi_label"
    if type_ == 'doc2vec':
        X_train, X_test, Y_train, Y_test, classes, n_classes, patent_ids = apply_doc2vec(data_frame, classification_type)
        # array - not numpy
        print('finisched apply doc2vec')
        vocab_processor, len_vocabulary = None, 0
    elif type_ == 'standard':
        standard_results = apply_standard_vectorizer(data_frame, classification_type)
        X_train, X_test, Y_train, Y_test, classes, n_classes, vocab_processor, len_vocabulary = standard_results
        patent_ids = data_frame['patent_id']
    else:
        Y_vect, classes, n_classes = apply_classification_vectorizer(classification_type, data_frame)
        vocab_processor, len_vocabulary = None, 0
        if type_ == 'tfidf_fit_transform': # not sure it remains the same order
            X_vect = pd.DataFrame({'text' : data_frame['text'], 'patent_id' : data_frame['patent_id']})
            X_train, X_test, Y_train, Y_test = get_train_test_from_data(X_vect, Y_vect)

            patent_ids = X_train['patent_id']
            print('almost started tfidf')
            vectorizer = apply_tfidf_vectorizer_fit(data_frame)
            print('applied tfidf')
            X_train = apply_tfidf_vectorizer_transform(X_train['text'], vectorizer)
            print('transformed train')
            X_test = apply_tfidf_vectorizer_transform(X_test['text'], vectorizer)
            print('transformed test')
            # csr matrix
        elif type_ == 'word2vec': # required google pre-trained vectors, it starts downloading if you don't have
            X_vect = apply_word2vec_word_averaging(data_frame)
            # ndarray
            print('finished word2vec word averaging')
            text_indexes = data_frame.index.values

            X_vect_temp = pd.DataFrame({'text' : text_indexes, 'patent_id' : data_frame['patent_id']})
            X_train_temp, X_test_temp, Y_train, Y_test = get_train_test_from_data(X_vect_temp, Y_vect)

            X_train, patent_ids = get_set_from_index(X_train_temp, X_vect)
            X_test, _ = get_set_from_index(X_test_temp, X_vect)
            print('finished word2vec vectorizer')
        else:
            data_frame.drop(columns=['classification'])
            X_train, X_test, Y_train, Y_test = get_train_test_from_data(data_frame, Y_vect)
            return X_train, X_test, Y_train, Y_test, classes, n_classes, vocab_processor, len_vocabulary
    save_training_set(patent_ids, model_name)
    return X_train, X_test, Y_train, Y_test, classes, n_classes, vocab_processor, len_vocabulary

def get_csv_path(model_key):
    root_location = get_root_location('data/')
    return link_paths(root_location, 'lstm_results.csv')
#     return 'content/drive/My\ Drive/Colab\ Notebooks/data/conv_results.csv'

def apply_method_to_create_metrics(tuple_):
    return list(tuple_) if tuple_ else 'None'

def get_parameters_values(parameters):
    parameters.replace(" ", "")
    return list(map(lambda token : token, parameters.split(' ')))

def get_parameters_list(model, metrics, parameters, classif_approach, classif_level, classif_type, dataset_location):
    list_ = [model]
    list_ += list(map(lambda item : item, metrics))
    list_ += [classif_approach, classif_level, classif_type]
    list_ += list(map(lambda item : item, parameters))
    list_ += [dataset_location]
    return list_

def get_metrics_values(metrics):
    return list(map(lambda tuple_ : apply_method_to_create_metrics(tuple_), metrics))

def get_sequential_LSTM_metrics_values(metrics):
    micro = ["precision:"+str(metrics["precision_micro"]), "recall:"+str(metrics["recall_micro"]), "f1:"+str(metrics["f1_micro"])]
    macro = ["precision:"+str(metrics["precision_macro"]), "recall:"+str(metrics["recall_macro"]), "f1:"+str(metrics["f1_macro"])]
    tops = ["top_1:"+str(metrics["top_1"]), "top_3:"+str(metrics["top_3"]), "top_5:"+str(metrics["top_5"])]
    return [str(metrics["coverage_error"]), micro, macro, tops]

def get_sequential_layers_values(parameters):
    return [list(map(lambda token : token, tokenize_text(parameters[:-1])))]

def write_dataframe_as_csv(data_frame, path_to_csv):
    if os.path.isfile(path_to_csv):
        with open(path_to_csv, 'a') as f:
            data_frame.to_csv(f, sep=',', header=False)
    else:
        data_frame.to_csv(path_to_csv, sep=',', header=True)

settings = {
    "Sequential" : {
                            "results_file_name" : "data/models_results/sequential_results.csv",
                            "metrics" : ['loss', 'accuracy', 'mse'],
                            "parameters" : ['layers'],
    },
    "Sequential Test" : {
                            "results_file_name" : "data/models_results/sequential_results.csv",
                            "metrics" : ['loss', 'accuracy', 'mse', 'micro', 'macro'],
                            "parameters" : ['layers'],
    },
    "Sequential Val" : {
                            "results_file_name" : "data/models_results/sequential_results.csv",
                            "metrics" : ['loss', 'accuracy', 'mse', 'micro', 'macro'],
                            "parameters" : ['layers'],
    }
}

def get_saving_dataframe(model_key, metrics_values, parameters_values, classif_approach, classif_level, classif_type, dataset_location):
    columns_list = get_parameters_list('model_name', settings[model_key]["metrics"], settings[model_key]["parameters"], 'approach', 'level', 'type', 'dataset_location')
    values_list = get_parameters_list(model_key, metrics_values, parameters_values, classif_approach, classif_level, classif_type, dataset_location)
    data_frame = pd.DataFrame(columns=columns_list)
    data_frame.loc[data_frame.shape[0] + 1] = values_list
    return data_frame

def save_results(model_key, metrics, parameters, classif_approach, classif_level, classif_type, dataset_location):
    print('###  saving_results  ###')
    path_to_csv = get_csv_path(model_key)

    if model_key == 'Sequential':
        metrics = list(map(lambda metric : [metric], metrics))
        # metrics = [[metric] for metric in metrics]
        metrics_values = get_metrics_values(metrics)
        parameters_values = get_sequential_layers_values(parameters)
    elif model_key == 'Sequential Test' or 'Sequential Val':
        metrics = [[-1], [-1], [-1], # metrics['loss'], metrics['accuracy'], metrics['mse'],
                   [metrics['precision_micro'], metrics['recall_micro'], metrics['f1_micro']],
                   [metrics['precision_macro'], metrics['recall_macro'], metrics['f1_macro']]]
        metrics_values = get_metrics_values(metrics)
        parameters_values = get_sequential_layers_values(parameters)
    elif model_key == 'FastText':
        metrics_values = metrics[1:3]
        parameters_values = [parameters]
    elif model_key == 'Sequential_LSTM':
        temp_metrics = metrics
        del temp_metrics["total_positive"], temp_metrics["average_num_of_labels"]
        metrics_values = get_sequential_LSTM_metrics_values(temp_metrics)
        parameters_values = get_sequential_layers_values(parameters)
    else:
        metrics_values = get_metrics_values(metrics)
        parameters_values = get_parameters_values(parameters)

    data_frame = get_saving_dataframe(model_key, metrics_values, parameters_values, classif_approach, classif_level, classif_type, dataset_location)
    write_dataframe_as_csv(data_frame, path_to_csv)

def get_sequential_classifier_information(model):
    text = str(model)
    string_list = []
    model.summary(print_fn=lambda x: string_list.append(x))
    layers = ' '.join(string_list)

    parameters = ""
    for index, el in enumerate(layers.split(' (')):
        if index > 0 and index < len(layers.split(' ('))-1:
            if len(el.split(' ')[-1]) >= 5:
                parameters += el.split(' ')[-1] + " "

    return text.split(' ')[0].split('.')[-1], parameters

def save_results_function(model_key, metrics, parameters, classif_approach, classif_level, classif_type, dataset_location):
    print('###  saving_results  ###')
    path_to_csv = get_csv_path(model_key)

    if model_key == 'Sequential':
        metrics = list(map(lambda metric : [metric], metrics))
        # metrics = [[metric] for metric in metrics]
        metrics_values = get_metrics_values(metrics)
        parameters_values = get_sequential_layers_values(parameters)
    elif model_key == 'Sequential Test' or 'Sequential Val':
        metrics = [[-1], [-1], [-1], # metrics['loss'], metrics['accuracy'], metrics['mse'],
                   [metrics['precision_micro'], metrics['recall_micro'], metrics['f1_micro']],
                   [metrics['precision_macro'], metrics['recall_macro'], metrics['f1_macro']]]
        metrics_values = get_metrics_values(metrics)
        parameters_values = get_sequential_layers_values(parameters)
    elif model_key == 'FastText':
        metrics_values = metrics[1:3]
        parameters_values = [parameters]
    elif model_key == 'Sequential_LSTM':
        temp_metrics = metrics
        del temp_metrics["total_positive"], temp_metrics["average_num_of_labels"]
        metrics_values = get_sequential_LSTM_metrics_values(temp_metrics)
        parameters_values = get_sequential_layers_values(parameters)
    else:
        metrics_values = get_metrics_values(metrics)
        parameters_values = get_parameters_values(parameters)

    data_frame = get_saving_dataframe(model_key, metrics_values, parameters_values, classif_approach, classif_level, classif_type, dataset_location)
    write_dataframe_as_csv(data_frame, path_to_csv)

def fill_matrix(data_matrix, source_dict, docs_list):
    """
    the use_get flag is for doc2vec_model.docvecs since it doesnt support .get(), so we catch the exception and
    fill with zeros in that case. This should really happen very rarely (if ever) so this exception handling
    should not be a drain on performance
    """
    for i, doc_id in enumerate(docs_list):
        child_ids = doc_id
        j = 0
        # try:
        if source_dict[child_ids] is not None:
            data_matrix[i][j] = source_dict[child_ids]
        else:
            data_matrix[i][j] = [0]
        # except:
        #     data_matrix[i][j] = [0]

def batch_generator(input_file, label_file, batch_size, QUEUE_SIZE, is_mlp=False, validate=False):
    q = Queue(maxsize=QUEUE_SIZE)
    p = ArrayReader(input_file, label_file, q, batch_size, is_mlp, validate)
    p.start()
    while True:
        item = q.get()
        if not item:
            p.terminate()
            print('Finished batch iteration')
            raise StopIteration()
        else:
            yield item

class ArrayReader(Process):
    def __init__(self, input_file, label_file, out_queue, batch_size, is_mlp=False, validate=False):
        super(ArrayReader, self).__init__()
        self.is_mlp = is_mlp
        self.validate = validate
        self.q = out_queue
        self.batch_size = batch_size
        self.input_file = input_file
        self.label_file = label_file

    def run(self):
        # x_file = np.load(self.input_file, mmap_mode='r')
        # y_file = np.load(self.label_file, mmap_mode='r')
        x_file = self.input_file
        y_file = self.label_file
        start_item = 0
        num_iter = 0
        while True:
            if start_item > y_file.shape[0]:
                # print('in new epoch for {}'.format(os.path.basename(self.input_file)))
                print('\nin new epoch for {}'.format(os.path.basename('X_data, y_train with validation data')))
                start_item = 0
            y_batch = y_file[start_item: start_item + self.batch_size]
            x_batch = x_file[start_item: start_item + self.batch_size]
            if self.is_mlp:
                x_batch = np.reshape(x_batch, (x_batch.shape[0], x_batch.shape[1] * x_batch.shape[2]))
            start_item += self.batch_size
            num_iter += 1
            try:
                self.q.put((x_batch, y_batch), block=True)
            except:
                return

In [0]:
# word model helper
class Word2VecHelper():
    def __init__(self):
        print('###  word2vec_vectorizer  ###')
        self.tokens = []
        self.mean = []

    def w2v_tokenize_text(self, text):
        self.tokens = []
        for word in tokenize_text(text):
            if len(word) < 2:
                continue
            self.tokens.append(word)
        return self.tokens

    def word_averaging(self, wv, words):
        all_words = set()
        self.mean = []

        for word in words:
            if isinstance(word, np.ndarray):
                self.mean.append(word)
            elif word in wv.vocab:
                self.mean.append(wv.syn0norm[wv.vocab[word].index])
                all_words.add(wv.vocab[word].index)

        if not self.mean:
            print("POSSIBLE ERROR IN WORD2VECHELPER: cannot compute similarity with no input %s", words)
            return np.zeros(wv.vector_size,)

        self.mean = gensim.matutils.unitvec(np.array(self.mean).mean(axis=0)).astype(np.float32)
        return self.mean

    def  word_averaging_list(self, wv, text_list):
        return np.vstack([self.word_averaging(wv, post) for post in text_list])

class Dov2VecHelper():
    def __init__(self):
        print('###  doc2vec_vectorizer  ###')
        self.labeled = []
        self.vectors = []

    def label_sentences(self, corpus, label_type):
        """
        Gensim's Doc2Vec implementation requires each document/paragraph to have a label associated with it.
        We do this by using the TaggedDocument method. The format will be "TRAIN_i" or "TEST_i" where "i" is
        a dummy index of the post.
        """
        self.labeled = []
        for i, v in enumerate(corpus):
            label = label_type + '_' + str(i)
            self.labeled.append(TaggedDocument(tokenize_text(v), [label]))
        return self.labeled

    def get_vectors(self, model, corpus_size, vectors_size, vectors_type):
        """
        Get vectors from trained doc2vec model
        :param doc2vec_model: Trained Doc2Vec model
        :param corpus_size: Size of the data
        :param vectors_size: Size of the embedding vectors
        :param vectors_type: Training or Testing vectors
        :return: list of vectors
        """
        self.vectors = np.zeros((corpus_size, vectors_size))
        for i in range(0, corpus_size):
            prefix = vectors_type + '_' + str(i)
            self.vectors[i] = model.docvecs[prefix]
        return self.vectors

def train_doc2vec(data_):
    cores = multiprocessing.cpu_count()
    model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, min_count=1, alpha=0.065, min_alpha=0.065, sample=0, workers=cores)
    model_dbow.build_vocab([x for x in tqdm(data_)])

    for epoch in range(30):
        model_dbow.train(utils_shuffle_rows([x for x in tqdm(data_)]), total_examples=len(data_), epochs=1)
        model_dbow.alpha -= 0.002
        model_dbow.min_alpha = model_dbow.alpha
    return model_dbow

In [0]:
# metrics helper
get_binary_0_5 = lambda x: 1 if x > 0.15 else 0
get_binary_0_5 = np.vectorize(get_binary_0_5)

metrics_graph_ranges = {
    'sections': {'min':0, 'max': 0.5},
    'classes': {'min':0, 'max': 0.05},
    'subclasses': {'min':0, 'max': 0.05}
}

def get_binary_classification(predictions, threshold):
    binary_predictions = lambda prediction : 1 if prediction > threshold else 0
    return np.vectorize(binary_predictions)

def display_sequential_metrics(algorithm, metrics):
    print('###  calculating_metrics  ###')
    print('@@@  ' + algorithm + '  @@@')
    print("Over all labels - Coverage error: {:.3f}, Average labels: {:.3f}".format(
        metrics['coverage_error'], metrics['average_num_of_labels']))
    print("Percentage - Top 1: {:.3f}, Top 3: {:.3f}, Top 5: {:.3f}".format(
        metrics['top_1'], metrics['top_3'], metrics['top_5']))
    print("Macro - precision: {:.3f}, recall: {:.3f}, f1: {:.3f}".format(
        metrics['precision_macro'], metrics['recall_macro'], metrics['f1_macro']))
    print("Micro - precision: {:.3f}, recall: {:.3f}, f1: {:.3f}".format(
        metrics['precision_micro'], metrics['recall_micro'], metrics['f1_micro']))

def get_top_N_percentage(y_score, y_true, max_N=3):
    """
    Get percentage of correct labels that are in the top N scores
    """
    num_all_true = 0
    num_found_in_max_N = 0
    for i in range(y_score.shape[0]):
        y_score_row = y_score[i, :]
        y_true_row = y_true[i, :]
        desc_score_indices = np.argsort(y_score_row)[::-1]
        true_indices = np.where(y_true_row ==1)[0]

        num_true_in_row = len(true_indices)
        num_all_true += num_true_in_row
        for i, score_index in enumerate(desc_score_indices):
            # only iterate through the score list till depth N, but make sure you also account for the case where
            # the number of true labels for the current row is higher than N
            if i >= max_N and i >= num_true_in_row:
                break
            if score_index in true_indices:
                num_found_in_max_N += 1
    return float(num_found_in_max_N) / num_all_true

def get_sequential_metrics(y_true, y_score, y_binary_score):
    """
    create the metrics object containing all relevant metrics
    """
    metrics = {}
    metrics['total_positive'] = np.sum(np.sum(y_binary_score))

    metrics['y_true'] = y_true
    metrics['y_score'] = y_score
    metrics['y_binary_score'] = y_binary_score

    metrics['coverage_error'] = coverage_error(y_true, y_score)
    metrics['average_num_of_labels'] = round(float(np.sum(np.sum(y_true, axis=1))) / y_true.shape[0], 2)

    metrics['average_precision_micro'] = average_precision_score(y_true, y_binary_score, average='micro')
    metrics['average_precision_macro'] = average_precision_score(y_true, y_binary_score, average='macro')

    metrics['precision_micro'] = precision_score(y_true, y_binary_score, average='micro')
    metrics['precision_macro'] = precision_score(y_true, y_binary_score, average='macro')
    metrics['recall_micro'] = recall_score(y_true, y_binary_score, average='micro')
    metrics['recall_macro'] = recall_score(y_true, y_binary_score, average='macro')
    metrics['f1_micro'] = f1_score(y_true, y_binary_score, average='micro')
    metrics['f1_macro'] = f1_score(y_true, y_binary_score, average='macro')

    # only calculate those for cases with a small number of labels (sections only)
    if y_true.shape[1] < 100:
        precision_scores = np.zeros(y_true.shape[1])
        for i in range(0, y_true.shape[1]):
            precision_scores[i] = precision_score(y_true[:,i], y_binary_score[:,i])
        metrics['precision_scores_array'] = precision_scores.tolist()

        recall_scores = np.zeros(y_true.shape[1])
        for i in range(0, y_true.shape[1]):
            recall_scores[i] = recall_score(y_true[:,i], y_binary_score[:,i])
        metrics['recall_scores_array'] = recall_scores.tolist()

        f1_scores = np.zeros(y_true.shape[1])
        for i in range(0, y_true.shape[1]):
            f1_scores[i] = f1_score(y_true[:,i], y_binary_score[:,i])
        metrics['f1_scores_array'] = f1_scores.tolist()

    metrics['top_1'] = get_top_N_percentage(y_score, y_true, max_N=1)
    metrics['top_3'] = get_top_N_percentage(y_score, y_true, max_N=3)
    metrics['top_5'] = get_top_N_percentage(y_score, y_true, max_N=5)
    return metrics

def get_data_files(base_location, classif_type, level, data_type):
    """
    get the files to load the data from for a certain classification type, level and data type
    """
    data_file = os.path.join(base_location, data_type_file_dict[data_type].format(level))
    labels_file = os.path.join(base_location, labels_type_file_dict[data_type].format(classif_type))
    return data_file, labels_file

def get_data(data_file, labels_file, mmap=False):
    """
    load np data with a certain mmap configuration
    """
    mmap_mode = None
    if mmap == True:
        mmap_mode = "r"
    X_data = np.load(data_file, mmap_mode=mmap_mode)
    y_data = np.load(labels_file, mmap_mode=mmap_mode)
    return X_data, y_data

def calculate_manual_metrics(algorithm, y_true, y_pred):
    a, b = y_true.T.copy(order = 'C'), y_pred.T.copy(order = 'C')
    
    print('//////// \ntrue_values:')
    print(a)
    print(a.shape)
    print('//////// \npredictions:')
    print(b)
    print(b.shape)

    count = 0
    count_tp, count_fp, count_tn, count_fn = 0, 0, 0, 0
    temp_count_tp, temp_count_fp, temp_count_tn, temp_count_fn = 0, 0, 0, 0
    temp_precision, temp_recall = [], []
    for y_test_el, y_pred_el in zip(np.nditer(a), np.nditer(b)):
        print(y_test_el, y_pred_el, end=', ')
        count += 1
        if y_pred_el == 1 and y_test_el == 1:
            temp_count_tp += 1
        if y_pred_el == 1 and y_test_el == 0:
            temp_count_fp += 1
        if y_pred_el == 0 and y_test_el == 0:
            temp_count_tn += 1
        if y_pred_el == 0 and y_test_el == 1:
            temp_count_fn += 1
        if count % y_true.shape[0] == 0:
            print(' ', count)
            if temp_count_tp+temp_count_fp != 0:
                precision = temp_count_tp/(temp_count_tp+temp_count_fp)
            else:
                precision = 0
            if temp_count_tp+temp_count_fn != 0:
                recall = temp_count_tp/(temp_count_tp+temp_count_fn)
            else:
                recall = 0
            temp_precision.append(precision)
            temp_recall.append(recall)

            count_tp += temp_count_tp
            count_fp += temp_count_fp
            count_tn += temp_count_tn
            count_fn += temp_count_fn

            temp_count_tp, temp_count_fp, temp_count_tn, temp_count_fn = 0, 0, 0, 0

    print('\nglobal - true positives : ', count_tp, 'true negatives : ', count_tn, 'false positives : ', count_fp, 'false negatives : ', count_fn)
    print('per class - precision values to average: ', temp_precision)
    print('per class - recall values to average: ', temp_recall)

    accuracy = (count_tp+count_tn)/(count_tp+count_tn+count_fp+count_fn)
    try:
        precision = count_tp/(count_tp+count_fp)
    except:
        precision = 0
    try:
        recall = count_tp/(count_tp+count_fn)
    except:
        recall = 0
    try:
        f1_score = 2*(precision*recall)/(precision+recall)
    except:
        f1_score = 0

    average_precision = sum(temp_precision)/len(temp_precision)
    average_recall = sum(temp_recall)/len(temp_recall)
    average_f1_score = 2*(average_precision*average_recall)/(average_precision+average_recall)

    print('test accuracy : {0:0.5f}'.format(accuracy))
    print('micro test precision : {0:0.5f},'.format(precision), ' macro test precision : {0:0.5f}'.format(average_precision))
    print('micro test recall : {0:0.5f},'.format(recall), ' macro test recall : {0:0.5f}'.format(average_recall))
    print('micro test f1_score : {0:0.5f},'.format(f1_score), ' macro test f1_score : {0:0.5f}'.format(average_f1_score))
    return [-1,-1,-1,-1], [-1,-1,-1,-1], [accuracy, precision, recall, f1_score], [accuracy, average_precision, average_recall, average_f1_score]

class MetricsCNNCallback(Callback):
    """
    Callback called by keras after each epoch. Records the best validation loss and periodically checks the
    validation metrics
    """
    def __init__(self, val_data, val_labels, patience):
        super(MetricsCNNCallback, self).__init__()
        self.val_data = val_data
        self.val_labels = val_labels

        self.patience = patience

        self.best_val_loss = None
        self.best_weights = None

    def on_train_begin(self, logs={}):
        self.epoch_index = 0

        self.wait = 0
        self.stopped_epoch = 0

        self.metrics_dict = {}
        self.best_val_loss = np.Inf
        self.best_weights = np.Inf
        self.best_validation_metrics = None

        self.losses = []
        self.val_losses = []

        self.val_predictions = None
        self.binary_predictions = None
        self.val_loss, self.val_acc, self.val_mse = 0, 0, 0
        self.validation_metrics = {}

    def on_epoch_end(self, epoch, logs=None):
        self.epoch_index += 1

        actual_val_loss = logs.get('val_loss')
        loss = logs.get('loss')
        self.losses.append(loss)
        self.val_losses.append(actual_val_loss)

        if np.less(actual_val_loss, self.best_val_loss):
            self.best_val_loss = actual_val_loss
            self.best_weights = self.model.get_weights()
            print('Found lower val loss for epoch {} => {}'.format(self.epoch_index, round(actual_val_loss, 5)))
        else:
            self.wait += 1
            if np.less(self.patience, self.wait):
                self.stopped_epoch = epoch
                self.model.stop_training = True
                print('Restoring model weights from the end of the best epoch.')
                self.model.set_weights(self.best_weights)

    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            print('Epoch %05d: early stopping' % (self.stopped_epoch + 1))

# class MetricsCallback():
class MetricsCallback(Callback):
    """
    Callback called by keras after each epoch. Records the best validation loss and periodically checks the
    validation metrics
    """
    def __init__(self, base_load_directory, classifications_type, level, batch_size, is_mlp=False):
        MetricsCallback.EPOCHS_BEFORE_VALIDATION = 10
        MetricsCallback.GRAPH_MIN = metrics_graph_ranges[classifications_type]['min']
        MetricsCallback.GRAPH_MAX = metrics_graph_ranges[classifications_type]['max']
        self.base_load_directory = base_load_directory
        self.classifications_type = classifications_type
        self.level = level
        self.batch_size = batch_size
        self.is_mlp = is_mlp

    def on_train_begin(self, logs={}):
        self.epoch_index = 0
        self.val_loss_reductions = 0
        self.metrics_dict = {}
        self.best_val_loss = np.iinfo(np.int32).max
        self.best_weights = None
        self.best_validation_metrics = None

        self.losses = []
        self.val_losses = []
        self.fig = plt.figure(figsize=(12,6), dpi=80)
        self.ax = plt.subplot(111)

    def on_epoch_end(self, epoch, logs={}):
        QUEUE_SIZE = 100
        self.epoch_index += 1
        self.losses.append(logs['loss'])
        self.val_losses.append(logs['val_loss'])
        loss_line, = self.ax.plot(range(1,self.epoch_index+1), self.losses, 'g-', label='Training Loss')
        val_loss_line, = self.ax.plot(range(1,self.epoch_index+1), self.val_losses, 'r-', label='Validation Loss')
        self.ax.legend(handles=[loss_line, val_loss_line])
        self.ax.set_ylim((MetricsCallback.GRAPH_MIN, MetricsCallback.GRAPH_MAX))
        self.fig.canvas.draw()
        if logs['val_loss'] < self.best_val_loss:
            self.val_loss_reductions += 1
            self.best_val_loss = logs['val_loss']
            self.best_weights = self.model.get_weights()
            print('\r    \r') # to remove the previous line of verbose output of model fit
            # time.sleep(0.1)
            print('Found lower val loss for epoch {} => {}'.format(self.epoch_index, round(logs['val_loss'], 5)))
            if self.val_loss_reductions % MetricsCallback.EPOCHS_BEFORE_VALIDATION == 0:

                print('Validation Loss Reduced {} times'.format(self.val_loss_reductions))
                print('Evaluating on Validation Data')
                Xv_file, yv_file = get_data_files(self.base_load_directory, self.classifications_type, self.level, 'validation') # creates the file paths
                Xv, yv = get_data(Xv_file, yv_file, mmap=True) # load the files as ndarray
                yvp = self.model.predict_generator(generator=batch_generator(Xv_file, yv_file,
                                                   self.batch_size, is_mlp=self.is_mlp, validate=True),
                                                   max_q_size=QUEUE_SIZE,
                                                   val_samples=yv.shape[1])
                yvp_binary = get_binary_0_5(yvp)
                print('Generating Validation Metrics')
                validation_metrics = get_sequential_metrics(yv, yvp, yvp_binary)
                print("****** Validation Metrics: Cov Err: {:.3f} | Top 3: {:.3f} | Top 5: {:.3f} | F1 Micro: {:.3f} | F1 Macro: {:.3f}".format(
                    validation_metrics['coverage_error'], validation_metrics['top_3'], validation_metrics['top_5'],
                    validation_metrics['f1_micro'], validation_metrics['f1_macro']))
                self.metrics_dict[self.epoch_index] = validation_metrics

In [0]:
# doc2vec sequential helper
def get_doc2vec_model(model_path):
    return Doc2Vec.load(model_path)

In [0]:
# predicting model helper
def plot_charts(history):
    from matplotlib import pyplot

    # plot loss during training
    pyplot.subplot(211)
    pyplot.title('Loss')
    pyplot.plot(history.history['loss'], label='train')
    pyplot.plot(history.history['val_loss'], label='test')
    pyplot.legend()

    # plot accuracy during training
    pyplot.subplot(212)
    pyplot.title('Accuracy')
    pyplot.plot(history.history['accuracy'], label='train')
    pyplot.plot(history.history['val_accuracy'], label='test')
    pyplot.legend()
    pyplot.show()

def get_lstm(optimizer,
            init_mode_1, activation_1,
            init_mode_2, activation_2,
            init_mode_3, activation_3,
            init_mode_4, activation_4,
            init_mode_5, activation_5,
            init_mode_6, activation_6,
            weight_constraint_1, 
            weight_constraint_2, 
            weight_constraint_3, 
            weight_constraint_4, 
            weight_constraint_5, 
            weight_constraint_6, 
            dropout_rate_1, 
            dropout_rate_2, 
            dropout_rate_3, 
            dropout_rate_4,
            neurons_1, neurons_2, neurons_3,
            filters_1, filters_2, filters_3,
            kernel_size_1, kernel_size_2, kernel_size_3,
            strides_1, strides_2, strides_3,
            activation_lstm_1, activation_lstm_2, activation_lstm_3,
            recurrent_activation_1, recurrent_activation_2, recurrent_activation_3,
            w_dropout_do_1, w_dropout_do_2, w_dropout_do_3, 
            u_dropout_do_1, u_dropout_do_2, u_dropout_do_3,
            backwards_1, backwards_2, backwards_3,
            unroll_1, unroll_2, unroll_3,
            lstm_output_size_1, lstm_output_size_2, lstm_output_size_3,
            input_size, n_classes):
    # len_vocabulary, n_classes = 64846, 302
    # len_vocabulary, n_classes = 9924, 62
    # build up the model

    stack_layers = 1

    # set up the layers
    model = Sequential()
    # model.add(keras.layers.Embedding(input_dim=len_vocabulary, output_dim=neurons_1))

    # model.add(keras.layers.ZeroPadding1D((1,1), input_shape=(len_vocabulary, neurons_1))) # (sequence_size, len_vocabulary) = X_data.shape[2], X_data.shape[0]
    # model.add(keras.layers.Conv1D(filters=filters_1, kernel_size=kernel_size_1, strides=strides_1, activation=activation_1, 
    #                               kernel_initializer=init_mode_1, kernel_constraint=tf.keras.constraints.max_norm(weight_constraint_1))) 
    # model.add(keras.layers.MaxPooling1D())

    # model.add(keras.layers.ZeroPadding1D((1,1), input_shape=(len_vocabulary, neurons_1)))
    # model.add(keras.layers.Conv1D(filters=filters_2, kernel_size=kernel_size_2, strides=strides_2, activation=activation_2, 
    #                               kernel_initializer=init_mode_2, kernel_constraint=tf.keras.constraints.max_norm(weight_constraint_2)))
    # model.add(keras.layers.GlobalMaxPooling1D())

    # model.add(keras.layers.ZeroPadding1D((1,1), input_shape=(len_vocabulary, neurons_1)))
    # model.add(keras.layers.Conv1D(filters=filters_3, kernel_size=kernel_size_3, strides=strides_3, activation=activation_3, kernel_initializer=init_mode_3, kernel_constraint=max_norm(weight_constraint_3)))
    # model.add(keras.layers.GlobalMaxPooling1D())

    model.add(LSTM(lstm_output_size_1, activation=activation_lstm_1, recurrent_activation=recurrent_activation_1, use_bias=False,
                   input_shape=(1, input_size), 
                   dropout_W=w_dropout_do_1, dropout_U=u_dropout_do_1,
                   implementation=1,
                   return_sequences=False if 1 == stack_layers else True,
                   go_backwards=backwards_1, stateful=False, unroll=unroll_1))
    
    # model.add(LSTM(lstm_output_size_2, activation=activation_lstm_2, recurrent_activation=recurrent_activation_2, use_bias=False,
    #                input_shape=(1, input_size), 
    #                dropout_W=w_dropout_do_2, dropout_U=u_dropout_do_2,
    #                implementation=1,
    #                return_sequences=False if 2 == stack_layers else True,
    #                go_backwards=backwards_2, stateful=False, unroll=unroll_2))

    # model.add(LSTM(lstm_output_size_3, activation=activation_lstm_3, recurrent_activation=recurrent_activation_3, use_bias=False,
    #                input_dim=len_vocabulary, dropout_W=w_dropout_do_3, dropout_U=u_dropout_do_3,
    #                implementation=1,
    #                return_sequences=False if 3 == stack_layers else True,
    #                go_backwards=backwards_3, stateful=False, unroll=unroll_3)

    # model.add(Flatten())
    # model.add(Dropout(dropout_rate_1))
    # model.add(Dense(neurons_1, activation=activation_4, kernel_initializer=init_mode_4, 
    #                 kernel_constraint=tf.keras.constraints.max_norm(weight_constraint_4)))
    # model.add(Dropout(dropout_rate_2))
    # model.add(Dense(neurons_2, activation=activation_5, kernel_initializer=init_mode_5, 
    #                 kernel_constraint=tf.keras.constraints.max_norm(weight_constraint_5)))
    # model.add(Dropout(dropout_rate_3))
    # model.add(Dense(neurons_3, activation=activation_6, kernel_initializer=init_mode_6, 
    #                 kernel_constraint=tf.keras.constraints.max_norm(weight_constraint_6)))
    # model.add(Dropout(dropout_rate_4))
    model.add(Dense(n_classes, activation='softmax'))

    model.summary()
    if optimizer == 'Adam':
        optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
    model.compile(optimizer=optimizer, 
                  # loss='binary_crossentropy', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy', 'mse'])
    return model

def run_lstm(model, 
             train_data, train_labels, test_data, test_labels, val_data, val_labels, 
             batch_size, callbacks_list, 
             train_flag,
             queue_size = 100,
             predicting_batch_size = 64, predicting_steps = None, predicting_max_queue_size = 100, predicting_callbacks_list = None, 
             epoch = 400):
    print('###  LSTM  ###')
    # compile the model

    if train_flag:
        # train the model
        history = model.fit_generator(generator=batch_generator(train_data, train_labels, batch_size, queue_size, is_mlp=False, validate=False),
                                      validation_data=batch_generator(val_data, val_labels, batch_size, queue_size, is_mlp=False, validate=True),
                                      # samples_per_epoch=len(train_data),
                                      samples_per_epoch=len(train_data) // batch_size,
                                      nb_val_samples=len(val_data),
                                      # nb_val_samples=len(val_data) // batch_size,
                                      nb_epoch=epoch,
                                      callbacks=callbacks_list,
                                      max_q_size=queue_size)
        # plot_charts(history)
        print('finished training')

    # using the recorded weights of the best recorded validation loss
    # last_model_weights = model.get_weights()
    # model.set_weights(metrics_callback.best_weights)

    # val_loss, val_acc, val_mse = model.evaluate_generator(generator=batch_generator(val_data, val_labels, predicting_batch_size, predicting_max_queue_size, is_mlp=False, validate=True),
    #                                                       max_queue_size=predicting_max_queue_size,
    #                                                       steps=len(val_data) // batch_size)

    test_loss, test_acc, test_mse = model.evaluate_generator(generator=batch_generator(test_data, test_labels, predicting_batch_size, predicting_max_queue_size, is_mlp=False, validate=True),
                                                             max_queue_size=predicting_max_queue_size,
                                                             steps=len(test_data) // batch_size)

    predictions = model.predict_generator(generator=batch_generator(test_data, test_labels, predicting_batch_size, predicting_max_queue_size, is_mlp=False, validate=True),
                                          max_q_size=predicting_max_queue_size,
                                          val_samples=len(test_data) // batch_size)
    pred_classes = model.predict_classes(test_data, verbose=0)
    print(pred_classes)
    
    print('finished predicting')

    # return history, predictions, binary_predictions
    # return [test_loss, test_acc, test_mse], predictions, [val_loss, val_acc, val_mse]
    return [test_loss, test_acc, test_mse], predictions, pred_classes, [None, None, None]
    return [None, None, None], predictions, pred_classes, [None, None, None]

def save_sequential_model(model, model_path):
    model.save(model_path)
    
def load_sequential_model(model_path):
    return load_model(model_path)

In [0]:
script_key = "lstm classify"

GLOBAL_VARS = namedtuple('GLOBAL_VARS', ['MODEL_NAME', 'DOC2VEC_MODEL_NAME', 'DOC2VEC_MODEL', 'NN_MODEL_NAME'])
NN_PARAMETER_SEARCH_PREFIX = "lstm_{}_level_{}_batch_{}_nn_parameter_searches.pkl"

def get_single_df(docs_list, sequence_size, EMBEDDING_SIZE, doc2vec_docvecs):
    data_ = np.ndarray((len(docs_list), sequence_size, EMBEDDING_SIZE), dtype=np.float32)
    fill_matrix(data_, doc2vec_docvecs, docs_list)
    return data_

def get_df_data(num_data, training_docs_list, val_docs_list, test_docs_list, sequence_size, EMBEDDING_SIZE, doc2vec_model_location):
    doc2vec_model = get_doc2vec_model(doc2vec_model_location)

    if num_data == 2:
        X_data = get_single_df(training_docs_list, sequence_size, EMBEDDING_SIZE, doc2vec_model.docvecs)
        Xv_data = get_single_df(val_docs_list, sequence_size, EMBEDDING_SIZE, doc2vec_model.docvecs)
        Xt_data = None
    elif num_data == 3:
        X_data = get_single_df(training_docs_list, sequence_size, EMBEDDING_SIZE, doc2vec_model.docvecs)
        Xv_data = get_single_df(val_docs_list, sequence_size, EMBEDDING_SIZE, doc2vec_model.docvecs)
        Xt_data = get_single_df(test_docs_list, sequence_size, EMBEDDING_SIZE, doc2vec_model.docvecs)
    return X_data, Xv_data, Xt_data

In [0]:
def get_classes_results(testy, yhat_classes, yhat_probs):
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score
    from sklearn.metrics import f1_score
    from sklearn.metrics import cohen_kappa_score
    from sklearn.metrics import roc_auc_score
    from sklearn.metrics import confusion_matrix

    print(testy.shape)
    print(yhat_classes.shape)
    print(yhat_probs.shape)
    try:
        # accuracy: (tp + tn) / (p + n)
        accuracy = accuracy_score(testy, yhat_classes)
        print('Accuracy: %f' % accuracy)
        # precision tp / (tp + fp)
        precision = precision_score(testy, yhat_classes)
        print('Precision: %f' % precision)
        # recall: tp / (tp + fn)
        recall = recall_score(testy, yhat_classes)
        print('Recall: %f' % recall)
        # f1: 2 tp / (2 tp + fp + fn)
        f1 = f1_score(testy, yhat_classes)
        print('F1 score: %f' % f1)
    except:
        try:
            # kappa
            kappa = cohen_kappa_score(testy, yhat_classes)
            print('Cohens kappa: %f' % kappa)
        except:
            try:
                # confusion matrix
                matrix = confusion_matrix(testy, yhat_classes)
                print(matrix)
            except:
                try:
                    # ROC AUC
                    auc = roc_auc_score(testy, yhat_probs)
                    print('ROC AUC: %f' % auc)
                except:
                    # accuracy: (tp + tn) / (p + n)
                    accuracy = accuracy_score(testy, yhat_probs)
                    print('Accuracy: %f' % accuracy)

def train_LSTM_from_web(data_frame, text_vectorizer, class_vectorizer, classif_level, classif_type, dataset_location):
    print('### LSTM - Training ###')

    date = datetime.now().isoformat()
    
    root_location = get_root_location('data/')
    sequential_model_location = link_paths(join_paths(root_location, "sequential_model/"), "sequential_lstm_model "+date)
    output_path = link_paths(join_paths(root_location, "model_results/"), "lstm_results.csv")
    losses_path = link_paths(join_paths(root_location, "training_losses/"), "lstm_losses.csv")
    
    save_results = True

    model_name = text_vectorizer+'/'+class_vectorizer+'/LSTM'
    results = apply_df_vectorizer(data_frame, text_vectorizer, class_vectorizer, model_name)
    X_train, X_test, train_labels, test_labels, classes, n_classes, vocab_processor, len_vocabulary = results

    X_train, X_val, train_labels, val_labels = get_train_test_from_data(X_train, train_labels)

    save_sets(join_paths(root_location, 'model_sets/'), X_train, X_test, X_val, train_labels, test_labels, val_labels, [classes, n_classes, vocab_processor, len_vocabulary])

    embedding_size = X_train.shape[1]

    if text_vectorizer == 'load_doc2vec':
        root_location = get_root_location('data/lstm_outcome/')
        doc2vec_vocab_location = link_paths(join_paths(root_location, "doc2vec_model/vocab_model/"), "doc2vec_vocab 2020-01-01")
        output_path = link_paths(join_paths(root_location, 'model_results/'), 'lstm_results.csv')

        training_docs_list = X_train['patent_id']
        val_docs_list = X_val['patent_id']
        test_docs_list = X_test['patent_id']

        sequence_size, embedding_size = 1, 200
        # train_data, val_data, _ = get_df_data(2, training_docs_list, val_docs_list, None, sequence_size, embedding_size, doc2vec_vocab_location)
        train_data, val_data, test_data = get_df_data(3, training_docs_list, val_docs_list, test_docs_list, sequence_size, embedding_size, doc2vec_vocab_location)

        vocab_processor = Doc2Vec.load(doc2vec_vocab_location)
        len_vocabulary = len(vocab_processor.wv.vocab)

    print('train_data: ', train_data[0])
    print('dataset shapes: ', train_data.shape, train_labels.shape, val_data.shape, val_labels.shape)
    
    len_vocabulary = 34736

    print("output path: ", output_path)
    print("length vocabulary: ", len_vocabulary, " number classes: ", n_classes)

    # best: {"batch_size": 64, "optimizer": "Adam", "init_mode_1": "uniform", "init_mode_2": "uniform", "init_mode_3": null, "init_mode_4": "uniform", "init_mode_5": "uniform", "init_mode_6": "uniform", "activation_1": "softmax", "activation_2": "softmax", "activation_3": null, "activation_4": "softmax", "activation_5": "softmax", "activation_6": "softmax", "weight_constraint_1": 3, "weight_constraint_2": 3, "weight_constraint_3": null, "weight_constraint_4": 3, "weight_constraint_5": 3, "weight_constraint_6": 3, "dropout_rate_1": 0.05, "dropout_rate_2": 0.1, "dropout_rate_3": 0.15, "dropout_rate_4": 0.2, "neurons_1": 910, "neurons_2": 273.0, "neurons_3": 182, "filters_1": 1, "filters_2": 1, "filters_3": null, "kernel_size_1": 1, "kernel_size_2": 1, "kernel_size_3": null, "strides_1": 1, "strides_2": null, "strides_3": null, "activation_lstm_1": "softmax", "activation_lstm_2": "softmax", "activation_lstm_3": "softmax", "recurrent_activation_1": "sigmoid", "recurrent_activation_2": "sigmoid", "recurrent_activation_3": "sigmoid", "w_dropout_do_1": 0.05, "w_dropout_do_2": 0.1, "w_dropout_do_3": 0.15, "u_dropout_do_1": 0.05, "u_dropout_do_2": 0.1, "u_dropout_do_3": 0.15, "backwards_1": true, "backwards_2": true, "backwards_3": true, "unroll_1": false, "unroll_2": false, "unroll_3": false, "lstm_output_size_1": 273, "lstm_output_size_2": null, "lstm_output_size_3": null}
    parameters = {"batch_size": 64,
                  "optimizer": "Adam",
                  "init_mode_1": "uniform",
                  "init_mode_2": "uniform",
                  "init_mode_3": None,
                  "init_mode_4": "uniform",
                  "init_mode_5": "uniform",
                  "init_mode_6": "uniform",
                  "activation_1": "softmax",
                  "activation_2": "softmax",
                  "activation_3": None,
                  "activation_4": "softmax",
                  "activation_5": "softmax",
                  "activation_6": "softmax",
                  "weight_constraint_1": 3,
                  "weight_constraint_2": 3,
                  "weight_constraint_3": None,
                  "weight_constraint_4": 3,
                  "weight_constraint_5": 3,
                  "weight_constraint_6": 3,
                  # "dropout_rate_1": 0.05,
                  # "dropout_rate_2": 0.1,
                  # "dropout_rate_3": 0.15,
                  # "dropout_rate_4": 0.2,
                  "dropout_rate_1": 0.0,
                  "dropout_rate_2": 0.0,
                  "dropout_rate_3": 0.0,
                  "dropout_rate_4": 0.0,
                  "neurons_1": 910,
                  "neurons_2": 273,
                  "neurons_3": 182,
                  "filters_1": 1,
                  "filters_2": 1,
                  "filters_3": None,
                  "kernel_size_1": 1,
                  "kernel_size_2": 1,
                  "kernel_size_3": None,
                  "strides_1": 1,
                  "strides_2": None,
                  "strides_3": None,
                  "activation_lstm_1": "softmax",
                  "activation_lstm_2": "softmax",
                  "activation_lstm_3": "softmax",
                  "recurrent_activation_1": "softmax",
                  "recurrent_activation_2": "softmax",
                  "recurrent_activation_3": "softmax",
                  # "w_dropout_do_1": 0.05,
                  # "w_dropout_do_2": 0.1,
                  # "w_dropout_do_3": 0.15,
                  # "u_dropout_do_1": 0.05,
                  # "u_dropout_do_2": 0.1,
                  # "u_dropout_do_3": 0.15,
                  "w_dropout_do_1": 0.05,
                  "w_dropout_do_2": 0.0,
                  "w_dropout_do_3": 0.0,
                  "u_dropout_do_1": 0.05,
                  "u_dropout_do_2": 0.0,
                  "u_dropout_do_3": 0.0,
                  "backwards_1": True,
                  "backwards_2": True,
                  "backwards_3": True,
                  "unroll_1": False,
                  "unroll_2": False,
                  "unroll_3": False,
                  "lstm_output_size_1": 939,
                  "lstm_output_size_2": 273,
                  "lstm_output_size_3": None
                }

    print("params: ", parameters)
    model = get_lstm(parameters['optimizer'], 
                    parameters['init_mode_1'], parameters['activation_1'],
                    parameters['init_mode_2'], parameters['activation_2'],
                    parameters['init_mode_3'], parameters['activation_3'],
                    parameters['init_mode_4'], parameters['activation_4'],
                    parameters['init_mode_5'], parameters['activation_5'],
                    parameters['init_mode_6'], parameters['activation_6'],
                    parameters['weight_constraint_1'], 
                    parameters['weight_constraint_2'], 
                    parameters['weight_constraint_3'], 
                    parameters['weight_constraint_4'], 
                    parameters['weight_constraint_5'], 
                    parameters['weight_constraint_6'], 
                    parameters['dropout_rate_1'], 
                    parameters['dropout_rate_2'], 
                    parameters['dropout_rate_3'], 
                    parameters['dropout_rate_4'],
                    parameters['neurons_1'], parameters['neurons_2'], parameters['neurons_3'],
                    parameters['filters_1'], parameters['filters_2'], parameters['filters_3'],
                    parameters['kernel_size_1'], parameters['kernel_size_2'], parameters['kernel_size_3'],
                    parameters['strides_1'], parameters['strides_2'], parameters['strides_3'],
                    parameters['activation_lstm_1'], parameters['activation_lstm_2'], parameters['activation_lstm_3'],
                    parameters['recurrent_activation_1'], parameters['recurrent_activation_2'], parameters['recurrent_activation_3'],
                    parameters['w_dropout_do_1'], parameters['w_dropout_do_2'], parameters['w_dropout_do_3'], 
                    parameters['u_dropout_do_1'], parameters['u_dropout_do_2'], parameters['u_dropout_do_3'],
                    parameters['backwards_1'], parameters['backwards_2'], parameters['backwards_3'],
                    parameters['unroll_1'], parameters['unroll_2'], parameters['unroll_3'],
                    parameters['lstm_output_size_1'], parameters['lstm_output_size_2'], parameters['lstm_output_size_3'],
                    embedding_size, n_classes)

    print('original: ', val_data.shape)
    try:
        val_data = val_data[val_data.index <= (val_data.shape[0] // parameters['batch_size']) * parameters['batch_size']]
        val_labels = val_labels[val_labels.index <= (val_labels.shape[0] // parameters['batch_size']) * parameters['batch_size']]
    except:
        val_data = val_data[:((val_data.shape[0] // parameters['batch_size']) * parameters['batch_size'])]
        val_labels = val_labels[:((val_labels.shape[0] // parameters['batch_size']) * parameters['batch_size'])]   
    print('shirnk: ', val_data.shape)

    min_delta, patience = 0.00001, 15
    early_stopper = EarlyStopping(monitor='val_loss', min_delta=min_delta, patience=patience, verbose=2, mode='auto')
    metrics_callback = MetricsCNNCallback(val_data, val_labels, patience)
    
    # learning rate scheduling
    # loss_history = LossHistory(parameters, losses_path)
    # # lrate = LearningRateScheduler(exp_decay)
    # lrate = LearningRateScheduler(step_decay)

    metrics, predictions, pred_classes, val_metrics = run_lstm(model, 
                                                               train_data, train_labels, val_data, val_labels, val_data, val_labels,
                                                               parameters['batch_size'], 
                                                               [early_stopper, metrics_callback],
                                                               # [loss_history, lrate, early_stopper, metrics_callback], 
                                                               True)
    save_sequential_model(model, sequential_model_location)
    print('model saved')    

    binary_predictions = get_binary_0_5(predictions)

    print('true values')
    print(val_labels)
    print(val_labels.shape)
    print(val_labels[0])
    print('predictions')
    print(predictions)
    print(predictions.shape)
    print(predictions[0])
    print('metrics - loss, acc, mse')
    print(metrics)
    # print('class predictions')
    # print(pred_classes)
    # print(pred_classes.shape)
    # print(pred_classes[0])
    print('binary predictions')
    print(binary_predictions)
    print(binary_predictions.shape)
    print(binary_predictions[0])

    # get_classes_results(val_labels, pred_classes, predictions)
    get_classes_results(val_labels, pred_classes, binary_predictions)
    
    classifier_name, parameters = get_sequential_classifier_information(model)
    model_name = text_vectorizer+'/'+class_vectorizer+'/'+classifier_name
    
    manual_metrics = calculate_manual_metrics(model_name, val_labels, binary_predictions)
    none_average, binary_average, micro_average, macro_average = manual_metrics

    validation_metrics = get_sequential_metrics(val_labels, predictions, binary_predictions)
    display_sequential_metrics(classifier_name, validation_metrics)
    
    if save_results:
        save_results_function(classifier_name+' Val', validation_metrics, parameters, model_name, classif_level, classif_type, dataset_location)

    return model   

In [0]:
def test_LSTM_from_web(model, data_frame, text_vectorizer, class_vectorizer, classif_level, classif_type, dataset_location):
    print('### LSTM Doing Testing ###')
    date = '01-01-2020'
    
    root_location = get_root_location('data/')
    sequential_model_location = link_paths(join_paths(root_location, "sequential_model/"), "sequential_lstm_model "+date)
    output_path = link_paths(join_paths(root_location, "model_results/"), "lstm_results.csv")
    losses_path = link_paths(join_paths(root_location, "training_losses/"), "lstm_losses.csv")
    
    model_name = text_vectorizer+'/'+class_vectorizer+'/lstm'
    # results = apply_df_vectorizer(data_frame, text_vectorizer, class_vectorizer, model_name)
    # X_data, Xt_data, y_train, y_test, classes, n_classes, vocab_processor, len_vocabulary = results

    # X_data, Xv_data, y_train, y_val = get_train_test_from_data(X_data, y_train)

    result_path = join_paths(root_location, "model_results")

    X_data, Xt_data, Xv_data, train_labels, test_labels, val_labels, settings = load_sets(join_paths(root_location, 'model_sets/'))

    embedding_size = X_data.shape[1]

    if text_vectorizer == 'load_doc2vec':
        root_location = get_root_location('data/lstm_outcome/')
        doc2vec_vocab_location = link_paths(join_paths(root_location, "doc2vec_model/vocab_model/"), "doc2vec_vocab 2020-01-01")
        result_path = join_paths(root_location, "model_results")
        output_path = link_paths(join_paths(root_location, "model_results/"), "lstm_results.csv")
        losses_path = link_paths(join_paths(root_location, "training_losses/"), "lstm_losses.csv")
        
        training_docs_list = X_data['patent_id']
        val_docs_list = Xv_data['patent_id']
        test_docs_list = Xt_data['patent_id']

        sequence_size, embedding_size = 1, 150
        train_data, val_data, test_data = get_df_data(3, training_docs_list, val_docs_list, test_docs_list, sequence_size, embedding_size, doc2vec_vocab_location)

        vocab_processor = Doc2Vec.load(doc2vec_vocab_location)
        len_vocabulary = len(vocab_processor.wv.vocab)
  
    save_results = True
    
    # train_data = train_data.reshape(train_data.shape[0], train_data.shape[2])
    # val_data = val_data.reshape(train_data.shape[0], train_data.shape[2])
    
    parameters = {"batch_size": 64}
    len_vocabulary = 57335
    len_vocabulary = 34736
    n_classes = train_labels.shape[1]

    print("output path: ", output_path)
    print("length vocabulary: ", len_vocabulary, " number classes: ", n_classes)
    
    # model = load_sequential_model(sequential_model_location)

    print('original: ', test_data.shape)
    try:
        test_data = test_data[test_data.index <= (test_data.shape[0] // parameters['batch_size']) * parameters['batch_size']]
        test_labels = test_labels[test_labels.index <= (test_labels.shape[0] // parameters['batch_size']) * parameters['batch_size']]
    except:
        test_data = test_data[:((test_data.shape[0] // parameters['batch_size']) * parameters['batch_size'])]
        test_labels = test_labels[:((test_labels.shape[0] // parameters['batch_size']) * parameters['batch_size'])]   
    print('shirnk: ', test_data.shape)

    metrics, predictions, _, test_metrics = run_lstm(model, 
                                                     train_data, train_labels, test_data, test_labels, test_data, test_labels,
                                                     parameters['batch_size'], None, 
                                                     False)
    binary_predictions = get_binary_0_5(predictions)
    
    classifier_name, parameters = get_sequential_classifier_information(model)
    model_name = text_vectorizer+'/'+class_vectorizer+'/'+classifier_name
    
    metrics = get_sequential_metrics(test_labels, predictions, binary_predictions)
    display_sequential_metrics(classifier_name, metrics)
    
    if save_results:
        save_results_function(classifier_name+' Test', metrics, parameters, model_name, classif_level, classif_type, dataset_location)


In [0]:
def shrink_vocabulary(row, vocabulary, data_frame, ids_list):
    if isinstance(row, pd.Series):
        patent_id, text, classification = row.tolist()

        new_tokens = [element for element in text if element in vocabulary]
        if new_tokens != [] or len(new_tokens) > 2:
            data_frame.loc[data_frame.shape[0]+1] = [patent_id, ' '.join(element for element in new_tokens), classification]
        else:
            ids_list.append(patent_id)

def further_preprocessing_phase(temp_data_frame):
    temp_data_frame['text'] = temp_data_frame['text'].apply(lambda text: th.tokenize_text(text) if text != None else '')
    # textlist = temp_data_frame['text'].to_numpy()
    textlist = temp_data_frame['text'].tolist()

    # if it raises an exeption could be the empty texts
    patent_dictionary = Dictionary(textlist)
    corpus = [patent_dictionary.doc2bow(text) for text in textlist]

    print('original dictionary size: ', len(patent_dictionary))

    vocab_tf={}
    for i in corpus:
        for item, count in dict(i).items():
            if item in vocab_tf:
                vocab_tf[item]+=int(count)
            else:
                vocab_tf[item] =int(count)

    remove_ids=[]
    no_of_ids_below_limit=0
    for id,count in vocab_tf.items():
        if count<=5:
            remove_ids.append(id)
    patent_dictionary.filter_tokens(bad_ids=remove_ids)

    patent_dictionary.filter_extremes(no_below=0)
    patent_dictionary.filter_n_most_frequent(30)

    print('parsed dictionary size: ', len(patent_dictionary))

    vocabulary = list(patent_dictionary.token2id.keys())

    ids_list = []
    data_frame = pd.DataFrame(columns=['patent_id', 'text', 'classification'])
    temp_data_frame.apply(lambda row : shrink_vocabulary(row, vocabulary, data_frame, ids_list), axis=1)
    print(len(ids_list))
    data_frame.set_index(data_frame['patent_id'], inplace=True)
    data_frame.drop(ids_list, axis=0, inplace=True)
    return data_frame

def shrink_to_sectors(classcode_list):
    new_list = []
    for class_ in classcode_list:
        if class_[0] not in new_list:
            new_list.append(class_[0])
    return ' '.join(element for element in new_list)

def shrink_classes(df, row, class_list):
    if isinstance(row, pd.Series):
        patent_id, text, class_ = row.tolist()
        new_classcodes = []
        classcodes = tokenize_text(class_)
        for classcode in classcodes:
            if not classcode in class_list:
                new_classcodes.append(classcode)
        if new_classcodes != []:
            new_class = ' '.join(el for el in new_classcodes)
            df.loc[df.shape[0] + 1] = [patent_id, text, new_class]

def reduce_amount_of_classes(data_frame, classification_df):
    if isinstance(classification_df, pd.DataFrame):
        threshold = int(data_frame.shape[0]/1000*0.35)
        if threshold == 0:
            threshold = 1
        temp_df = classifications_df[classifications_df['count'] <= threshold]
        classes_list = temp_df['class'].tolist()
    elif isinstance(classification_df, list):
        classes_list = classification_df
    df = pd.DataFrame(columns=['patent_id', 'text', 'classification'])
    data_frame.apply(lambda row : shrink_classes(df, row, classes_list), axis=1)
    return df, classes_list

if __name__ == '__main__':
    try:
        if len(sys.argv) == 2:
            # source_path = 'test_clean/*/directories - and inside all the patents'
            source_path = sys.argv[1]

            # here the source_path must be passed as a string of the root directory of all data folders!
            source_path, folder_level = handle_partial_args(source_path)
        else:
            print(usage())
            sys.exit(1)
    except:
        # source_path = ['/Users/elio/Desktop/Patent-Classification/data/test_classification/cleaned/B/']
        source_path = ['/Users/elio/Desktop/Patent-Classification/data/test_classification/cleaned/B - 1500 patents/']
        source_path = ['/content/drive/My Drive/Colab Notebooks/data/test_classification/cleaned/B - 1500 patents/']
    
    start = time.time()
    print(source_path)
    
    text_vectorizer = 'load_data/load_doc2vec'
    class_vectorizer = 'multi_label'
    classif_level = 'description'
    classif_type = 'subclasses'

    # if there is the '/' into the text_vectorizer, you are supposed to load data from csv
    index = text_vectorizer.rfind('/')
    if index == -1:
        # first option: here you read the directory with the patents
        patent_ids, temp_df, classifications_df = load_data(source_path)
        data_frame, classif_level, classif_type = get_final_df(patent_ids, temp_df, classif_type)

        # data_frame, _ = reduce_amount_of_classes(data_frame, classifications_df)

        # save_data_frame(script_key, data_frame, csvfile)
    else:
        # second option: here you specify the csv
        csvfile = 'training_2000_2004_cleaned.csv'
        csvfile = 'training_2000_2004_cleaned_test.csv'
        csvfile = 'training_2000_2004_cleaned_test_middle.csv'
        # save_data_frame(script_key, data_frame, csvfile)
        classif_level, classif_type = 'description_claim_abstract_title', 'subclasses'
        training_df, train_classifications_df = load_data_frame(script_key, csvfile)
        
        csvfile = 'testing_2005_cleaned.csv'
        csvfile = 'testing_2005_cleaned_test.csv'
        csvfile = 'testing_2005_cleaned_test_smaller.csv'
        testing_df, classifications_df = load_data_frame(script_key, csvfile)
        
        # train, classes_to_remove = reduce_amount_of_classes(training_df, train_classifications_df)
        # test, _ = reduce_amount_of_classes(testing_df, classes_to_remove)

        data_frame = pd.concat([training_df, testing_df])
        patent_ids = data_frame['patent_id'].tolist()

        text_vectorizer = text_vectorizer[index+1:]

    # around 50.000
    # data_frame = further_preprocessing_phase(data_frame)

    # data_frame['classification'] = data_frame['classification'].apply(lambda classcode : shrink_to_sectors(tokenize_text(classcode)))
    # classification_df = pd.DataFrame(columns=['class', 'count'])
    # data_frame['classification'].apply(lambda classcode : calculate_class_distribution(classcode, classification_df))
    # print('sectors distribution : \n', classification_df)

    print('dataframe shape: ', data_frame.shape)

    model = train_LSTM_from_web(data_frame, text_vectorizer, class_vectorizer, classif_level, classif_type, source_path)
    
    # this would test the model on testing set
    # test_LSTM_from_web(model, data_frame, text_vectorizer, class_vectorizer, classif_level, classif_type, source_path)

    print("end lstm classifition step, execution time: ", time.time()-start)

# TODO list:
# try all the state-of-the art and maybe we can combine two of them.
#
# turn on the validation settings
#
# get_lstm_shapes, try with validation data. change batch size. change testing parameters according to training results

['/content/drive/My Drive/Colab Notebooks/data/test_classification/cleaned/B - 1500 patents/']
dataframe shape:  (6427, 3)
ids_list:  []
dataframe shape:  (6427, 3)
dataframe shape:  (20, 3)
ids_list:  []
dataframe shape:  (20, 3)
dataframe shape:  (6447, 3)
### LSTM - Training ###
###  multi_label_binarizer  ###


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


train_data:  [[-0.14926393  0.07977636 -0.38661826 -0.07619986  0.14820573  0.3177496
   0.26178342  0.46967736  0.17745563  0.38124922  0.2027556   0.10479432
   0.50334734 -0.04895656 -0.24495538  0.11185762  0.13570793  0.2687881
  -0.27668673  0.12726595 -0.4208606   0.08846858 -0.45114213  0.25289002
   0.30377838  0.17314757  0.2933514  -0.15407667  0.05223534 -0.15608248
   0.41895208  0.34857622  0.10931011  0.10909563  0.07831977 -0.03037874
   0.19191973  0.28977036 -0.2738388   0.45971164  0.24957372  0.16481625
  -0.16774483 -0.55281603 -0.36570773  0.02722728  0.15667216 -0.10197079
   0.03612706 -0.17816657 -0.33224815 -0.07364052 -0.09939025  0.02899808
   0.23298644 -0.26179814  0.12099829 -0.06969366 -0.2682944   0.18446282
  -0.27763772 -0.37492537 -0.06047355 -0.17538446  0.06546511  0.06101125
  -0.1253456  -0.48712656 -0.06078298 -0.14184383  0.32122707 -0.11701781
  -0.3812059  -0.24231191 -0.38690302 -0.0822603  -0.58484185  0.3014285
  -0.26212066 -0.05576542 -0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:77: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(939, activation="softmax", recurrent_activation="softmax", use_bias=False, input_shape=(1, 200), implementation=1, return_sequences=False, go_backwards=True, stateful=False, unroll=False, dropout=0.05, recurrent_dropout=0.05)`



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 939)               4278084   
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 7520      
Total params: 4,285,604
Trainable params: 4,285,604
Non-trainable params: 0
_________________________________________________________________


original:  (1032, 1, 200)
shirnk:  (1024, 1, 200)
###  LSTM  ###
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


...

Streaming output truncated to the last 5000 lines.

in new epoch for X_data, y_train with validation data

in new epoch for X_data, y_train with validation data

in new epoch for X_data, y_train with validation data

in new epoch for X_data, y_train with validation data

in new epoch for X_data, y_train with validation data

in new epoch for X_data, y_train with validation data

in new epoch for X_data, y_train with validation data

in new epoch for X_data, y_train with validation data

in new epoch for X_data, y_train with validation data
64/64 [==============================] - 32s 495ms/step - loss: 2.4013 - acc: 0.2577 - mean_squared_error: 0.1268 - val_loss: nan - val_acc: nan - val_mean_squared_error: nan
Epoch 11/400
 3/64 [>.............................] - ETA: 10s - loss: 2.4266 - acc: 0.2708 - mean_squared_error: 0.1279
in new epoch for X_data, y_train with validation data
63/64 [============================>.] - ETA: 0s - loss: 2.3941 - acc: 0.2571 - mean_squared_error: 0.12

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:151: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:151: UserWarning: Update your `predict_generator` call to the Keras 2 API: `predict_generator(generator=<generator..., steps=16, max_queue_size=100)`



in new epoch for X_data, y_train with validation data

in new epoch for X_data, y_train with validation data

in new epoch for X_data, y_train with validation data

in new epoch for X_data, y_train with validation data

in new epoch for X_data, y_train with validation data

in new epoch for X_data, y_train with validation data

in new epoch for X_data, y_train with validation data

in new epoch for X_data, y_train with validation data

in new epoch for X_data, y_train with validation data

in new epoch for X_data, y_train with validation data

in new epoch for X_data, y_train with validation data

in new epoch for X_data, y_train with validation data
[0 0 0 ... 0 0 0]
finished predicting
model saved
true values
[[0 0 0 ... 0 0 1]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 1 1]
 [0 0 0 ... 0 1 0]
 [0 0 1 ... 0 0 0]]
(1024, 8)
[0 0 0 0 0 0 0 1]
predictions
[[0.20440532 0.15428013 0.17565835 ... 0.06381655 0.1840171  0.17374788]
 [0.20439747 0.1542844  0.17565727 ... 0.0638

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


###  calculating_metrics  ###
@@@  Sequential  @@@
Over all labels - Coverage error: 3.578, Average labels: 1.270
Percentage - Top 1: 0.270, Top 3: 0.567, Top 5: 0.893
Macro - precision: 0.142, recall: 0.625, f1: 0.231
Micro - precision: 0.227, recall: 0.893, f1: 0.361
###  saving_results  ###
end lstm classifition step
